In [6]:
import sqlite3  # 这就是我们的数据库引擎
import pandas as pd  # 处理表格数据的神器
import numpy as np  # 处理数值的帮手
import matplotlib.pyplot as plt  # 画图用的，后面会用到
# %matplotlib inline  # 让图表直接在Notebook里显示

# 2. 建立一个数据库连接。如果不存在 'online_retail.db' 这个文件，它会自动创建。
# 把它想象成你打开了数据库软件，准备干活。
conn = sqlite3.connect('online_retail.db')
print("✅ 数据库连接已建立！")

✅ 数据库连接已建立！


In [7]:
# 用来查看文件的编码文件
import chardet

# 检测文件编码
with open('OnlineRetail.csv', 'rb') as f:
    result = chardet.detect(f.read())

print(f"检测到的编码是: {result['encoding']}")

# 使用检测到的编码读取文件
df = pd.read_csv('OnlineRetail.csv', encoding=result['encoding'])
print('查看数据形状:', df.shape)
print('前5行:')
display(df.head())

检测到的编码是: ISO-8859-1
查看数据形状: (541909, 8)
前5行:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


- encoding='unicode_escape'：这个数据集编码有点怪，不加这个参数你很可能看到乱码报错，记住它，这是经验。

- parse_dates=['InvoiceDate']：主动告诉 pandas 把 “InvoiceDate” 列当作日期时间类型处理，后续做时间筛选时你会感谢我这个提醒。

- 运行后，你应该能看到数据预览和一个成功的提示。这时，你目录下会生成一个 online_retail.db 文件，这就是你的数据库了。

- conn这是一个数据库连接对象，它代表了与特定数据库的连接。这个对象需要在调用 to_sql 方法之前创建，并传递给该方法，以便 to_sql 方法知道要将数据存储到哪个数据库中

In [13]:
# 3. 用 pandas 读取 CSV 文件。注意编码和日期格式，这个数据集有坑，我们提前绕过。
# 把 ‘你的文件路径’ 这几个字，替换成你电脑上 OnlineRetail.csv 的真实路径，例如：r'C:\Users\你的名字\Downloads\OnlineRetail.csv'
# 注意：路径前的 ‘r’ 是为了防止转义字符出错，Windows用户尤其要注意！
file_path = 'OnlineRetail.csv'
df = pd.read_csv(file_path, encoding='unicode_escape', parse_dates=['InvoiceDate'])

# 4. 看一眼数据长什么样，这是数据分析师的本能。head()是看前5行。
print("📄 数据预览 (前5行):")
print(df.head())
print("\n📊 数据形状 (行数，列数):", df.shape)

# 5. 将 pandas DataFrame 写入 SQLite 数据库，命名为 ‘retail’ 表。
# if_exists='replace' 意思是：如果已经有叫 ‘retail’ 的表，就替换掉。第一次跑放心用。
df.to_sql('retail', conn, if_exists='replace', index=False)
print("🎉 数据已成功导入数据库表 ‘retail’ ！")

📄 数据预览 (前5行):
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  

📊 数据形状 (行数，列数): (541909, 8)
🎉 数据已成功导入数据库表 ‘retail’ ！


# 1.数据提取与基本探索

- 目标：熟悉数据，练习SELECT, WHERE, COUNT, DISTINCT，日期函数。
- 关键：准确筛选和汇总数据。

#### 任务1.1：基础查询：从数据集中提取指定国家（如德国）上个月的所有订单数据。

- SELECT *：在探索阶段常用，但实际工作中和面试时，严禁在正式代码或报告里写 SELECT *。你必须明确指明需要的字段。这里只是为了演示。

- WHERE：这是过滤数据的生命线。面试必考。注意，在SQL中，字符串用单引号。

- COUNT(*) 和 COUNT(DISTINCT ...)：这是两个天差地别的概念。

- COUNT(*)：数的是总行数。

- COUNT(DISTINCT CustomerID)：数的是不重复的客户ID数量。如果客户有重复购买，他只被计算一次。面试官100%会问你它们的区别。

- MIN() / MAX()：聚合函数，这里用来确定数据的时间范围。

In [15]:
# 6.创建一个游标,这个向数据库发送sql指令的'传话筒'
cur = conn.cursor()

# 执行SQL查询
query_task1_1 = """
SELECT *
FROM retail
WHERE Country == 'Germany'
-- LIMIT 10; -- 初次查询，先用LIMIT看看结果，确认无误后再去掉它运行全部数据
"""
# 先只取10条看看，没问题就注释掉 LIMIT 10，再运行一次
df_task1_1 = pd.read_sql_query(query_task1_1,conn)
print(f'德国地区共有{len(df_task1_1)}条交易记录')
display(df_task1_1)

德国地区共有9495条交易记录


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany
1,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany
2,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany
3,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany
4,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
9490,581578,22993,SET OF 4 PANTRY JELLY MOULDS,12,2011-12-09 12:16:00,1.25,12713.0,Germany
9491,581578,22907,PACK OF 20 NAPKINS PANTRY DESIGN,12,2011-12-09 12:16:00,0.85,12713.0,Germany
9492,581578,22908,PACK OF 20 NAPKINS RED APPLES,12,2011-12-09 12:16:00,0.85,12713.0,Germany
9493,581578,23215,JINGLE BELL HEART ANTIQUE SILVER,12,2011-12-09 12:16:00,2.08,12713.0,Germany


#### 任务1.2：数据概览 - 计算核心指标

Invoice-----发票

Customer----客户

Stock----库存

distinct----独特的

In [20]:
query_task1_2 = """
SELECT
    COUNT(*) as 总订单行数,
    COUNT(DISTINCT InvoiceNo) as 唯一订单数,
    COUNT(DISTINCT CustomerID) as 唯一客户数,
    COUNT(DISTINCT StockCode) as 唯一商品种类,
    MIN(InvoiceDate) as 最早订单日期,
    MAX(InvoiceDate) as 最晚订单日期
FROM retail;
"""
df_task1_2 = pd.read_sql_query(query_task1_2,conn)
# print(df_task1_2)
df_task1_2

,总订单行数,唯一订单数,唯一客户数,唯一商品种类,最早订单日期,最晚订单日期
0,541909,25900,4372,4070,2010-12-01 08:26:00,2011-12-09 12:50:00


#### 1.3更深入的探索(练习)

找出每个国家的销售额排名。这涉及到 GROUP BY、SUM 和排序。
- 发票编号-股票代码-描述-数量-发票日期-单价-客户ID-国家
- WHERE Quantity > 0：这是数据清洗思维！ 原始数据里，退货订单用负数表示。如果你不排除，总销售额会被拉低。在面试中展示出你能主动想到这一点，是巨大的加分项。

- GROUP BY 和 ORDER BY：GROUP BY 后面跟着分组的字段（这里是国家），ORDER BY 决定结果的展示顺序。

In [24]:
print('不同国家销售额排名:')
query_task1_3 = """
SELECT
    Country,
    COUNT(DISTINCT InvoiceNo) AS 订单数,
    SUM(Quantity * UnitPrice) AS 销售额
FROM retail
WHERE Quantity > 0
GROUP BY Country
ORDER BY 销售额 DESC
LIMIT 10;
"""
df_task1_3 = pd.read_sql_query(query_task1_3,conn)
df_task1_3

不同国家销售额排名:


,Country,订单数,销售额
0,United Kingdom,18786,9003097.964
1,Netherlands,95,285446.340
2,EIRE,288,283453.960
3,Germany,457,228867.140
4,France,392,209715.110
5,Australia,57,138521.310
6,Spain,90,61577.110
7,Switzerland,54,57089.900
8,Belgium,98,41196.340
9,Sweden,36,38378.330


#### 5.收尾与好习惯

所有任务跑完，必须关闭数据库连接，释放资源。这是职业素养。

In [25]:
# 7. 关闭游标和数据库连接（非常重要！）
cur.close()
conn.close()
print("🔒 数据库连接已关闭。")

🔒 数据库连接已关闭。


#### 练习:把任务1.1的SQL语句去掉 LIMIT 10，再运行一次，感受数据量。

In [10]:
cur = conn.cursor()
query_task1_1 = """
SELECT *
FROM retail
WHERE Country = 'Sweden';
"""
df_task1_1 = pd.read_sql_query(query_task1_1,conn)
print(f'瑞典地区共有{len(df_task1_1)}条交易记录')
print(df_task1_1)

瑞典地区共有462条交易记录
    InvoiceNo StockCode                        Description  Quantity  \
0     C538847    85232B    SET OF 3 BABUSHKA STACKING TINS      -240   
1      538848    85232B    SET OF 3 BABUSHKA STACKING TINS       240   
2      539338     84077  WORLD WAR 2 GLIDERS ASSTD DESIGNS       576   
3      539338     22951    60 CAKE CASES DOLLY GIRL DESIGN       240   
4      539338     22417     PACK OF 60 SPACEBOY CAKE CASES       240   
..        ...       ...                                ...       ...   
457    578089     21731      RED TOADSTOOL LED NIGHT LIGHT        12   
458    578089      POST                            POSTAGE         4   
459   C578090      POST                            POSTAGE        -2   
460    580704     23494   VINTAGE DOILY DELUXE SEWING KIT         40   
461   C580702     23494   VINTAGE DOILY DELUXE SEWING KIT        -40   

             InvoiceDate  UnitPrice  CustomerID Country  
0    2010-12-14 13:27:00       4.95     17404.0  Sweden  
1   

#### 修改任务1.1，查询 Country = 'France' 且 Quantity > 5 的所有记录。这是模拟面试官给你加条件。

In [12]:
query_task1_4 = """
SELECT *
FROM retail
WHERE Country = 'France' 
AND Quantity > 5 ;
"""
df_task1_4 = pd.read_sql_query(query_task1_4,conn)
print(f'法国订单数大于5的一共有{len(df_task1_4)}条记录')
df_task1_4

法国订单数大于5的一共有6328条记录


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-12-01 08:45:00,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-12-01 08:45:00,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,2010-12-01 08:45:00,0.65,12583.0,France
...,...,...,...,...,...,...,...,...
6323,581587,22555,PLASTERS IN TIN STRONGMAN,12,2011-12-09 12:50:00,1.65,12680.0,France
6324,581587,22367,CHILDRENS APRON SPACEBOY DESIGN,8,2011-12-09 12:50:00,1.95,12680.0,France
6325,581587,22629,SPACEBOY LUNCH BOX,12,2011-12-09 12:50:00,1.95,12680.0,France
6326,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France


In [22]:
# 快速侦察：查看数据库中有哪些表，以及retail表的前几个字段
quick_check = pd.read_sql_query("SELECT * FROM retail LIMIT 0;", conn)
print("✅ 表 'retail' 连接成功。")
print("该表包含的列有：", quick_check.columns.tolist())
print("'发票编号', '库存代码', '商品描述', '数量', '发票日期', '单价', '客户ID', '国家'")

✅ 表 'retail' 连接成功。
该表包含的列有： ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
'发票编号', '库存代码', '商品描述', '数量', '发票日期', '单价', '客户ID', '国家'


#### 模仿加分任务，写一段SQL，计算每个 CustomerID 的总购买金额，并找出消费最高的前5名客户。（提示：GROUP BY CustomerID）

In [16]:
query_task1_5 = """
SELECT 
    CustomerID AS 客户id,
    SUM(Quantity*UnitPrice) AS 购买金额
FROM retail
GROUP BY CustomerID
ORDER BY 购买金额 DESC
LIMIT 5;
"""
df_task1_5= pd.read_sql_query(query_task1_5,conn)
df_task1_5

,客户id,购买金额
0,NaN,1447682.12
1,14646.0,279489.02
2,18102.0,256438.49
3,17450.0,187482.17
4,14911.0,132572.62


#### 2.1时间序列分析：

计算2011年每月的销售额和环比增长率（面试高频）
- strftime 函数，该函数通常在 SQLite 数据库中用来格式化日期和时间
- LAG 是一个窗口函数，用于访问当前行的前一行的数据。在这个例子中，它获取前一个月的总销售额。
- OVER (ORDER BY strftime('%Y-%m', InvoiceDate)) 指定了窗口的排序规则，即按照 InvoiceDate 字段格式化后的年月顺序进行排序。

In [21]:
query_task2_1 = """
SELECT 
    strftime('%Y-%m',InvoiceDate) as 年月,
    SUM(Quantity*UnitPrice) as 当月销售额,
    (SUM(Quantity*UnitPrice)-LAG(SUM(Quantity*UnitPrice)) OVER (ORDER BY strftime('%Y-%m',InvoiceDate)))
    / LAG(SUM(UnitPrice * Quantity)) OVER (ORDER BY strftime('%Y-%m', InvoiceDate))*100 as 环比增长率
FROM retail
WHERE Quantity > 0 AND InvoiceDate >= '2011-01-01' AND InvoiceDate <= '2012-01-01'
GROUP BY 年月
ORDER BY 年月;
"""
df_task2_1 = pd.read_sql_query(query_task2_1,conn)
df_task2_1

,年月,当月销售额,环比增长率
0,2011-01,691364.560,NaN
1,2011-02,523631.890,-24.261103
2,2011-03,717639.360,37.050354
3,2011-04,537808.621,-25.058650
4,2011-05,770536.020,43.273274
5,2011-06,761739.900,-1.141559
6,2011-07,719221.191,-5.581788
7,2011-08,737014.260,2.473936
8,2011-09,1058590.172,43.632251
9,2011-10,1154979.300,9.105424


#### 用户分层（RFM模型雏形）

基于数据集计算每个客户的最近购买日、购买频率和总金额
- 最近购买日---max(InvoiceDate)
- 购买频率---count(DISTINCT InvoiceNo)
- 总金额---sum(Quantity*UnitPrice)
- WHERE Quantity > 0 AND CustomerID IS NOT NULL(要自己记得)

In [26]:
query_task2_2 = """
SELECT
    CustomerID,
    MAX(InvoiceDate) as 最近购买时间,
    COUNT(DISTINCT InvoiceNo) as 购买频率,
    SUM(Quantity*UnitPrice) as 总金额
FROM retail
WHERE Quantity > 0 AND CustomerID IS NOT NULL
GROUP BY CustomerID;
"""
df_task2_2 = pd.read_sql_query(query_task2_2,conn)
df_task2_2

,CustomerID,最近购买时间,购买频率,总金额
0,12346.0,2011-01-18 10:01:00,1,77183.60
1,12347.0,2011-12-07 15:52:00,7,4310.00
2,12348.0,2011-09-25 13:13:00,4,1797.24
3,12349.0,2011-11-21 09:51:00,1,1757.55
4,12350.0,2011-02-02 16:01:00,1,334.40
...,...,...,...,...
4334,18280.0,2011-03-07 09:52:00,1,180.60
4335,18281.0,2011-06-12 10:53:00,1,80.82
4336,18282.0,2011-12-02 11:43:00,2,178.05
4337,18283.0,2011-12-06 12:02:00,16,2094.88
